In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim  

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
geocoder = Nominatim(user_agent='my-application', timeout=2)

# Gun Violence

## Load

In [2]:
df_gv = pd.read_csv('data/gun-violence-data_01-2013_03-2018.csv', index_col='incident_id')
df_gv.head()

,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
incident_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


## To Do
* Drop
    * source_url
    * incident_url_fields_missing
    * congressional_district
    * gun_stolen
    * incident_characteristics
    * n_guns_involved
    * sources
    * state_house_district
    * state_senate_district
* Dates
    * datetime
    * Delete pre-2014 data
* Clean
    * gun_type
    * participant_age
    * participant_age_group
    * participant_gender
    * participant_name
    * participant_relationship
    * participant_status
    * participant_type

## Drop

In [3]:
drop_cols = [
    'source_url',
    'incident_url_fields_missing',
    'congressional_district',
    'gun_stolen',
    'incident_characteristics',
    'n_guns_involved',
    'sources',
    'state_house_district',
    'state_senate_district'
]

df_gv.drop(columns=drop_cols, inplace=True)

## Clean

In [4]:
df_gv['date'] = pd.to_datetime(df_gv['date'])
df_gv = df_gv[df_gv['date'].apply(lambda x: x.year >= 2014)]

In [5]:
def cleaner(string):
    '''
    Returns a list of cleaned strings; broken down from the input string.
    '''
    if type(string) != str:
        return string
    else: 
        cleaned_list = [word[3:] for word in string.split('||')]
        cleaned_string = ','.join(cleaned_list)
        return cleaned_string

In [6]:
clean_cols = [
    'gun_type',
    'participant_age',
    'participant_age_group',
    'participant_gender',
    'participant_name',
    'participant_relationship',
    'participant_status',
    'participant_type'
]

for col in clean_cols:
    df_gv[col] = df_gv[col].apply(cleaner, convert_dtype=False)

## Save

In [7]:
df_gv.head()

,date,state,city_or_county,address,n_killed,n_injured,incident_url,gun_type,latitude,location_description,longitude,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type
incident_id,,,,,,,,,,,,,,,,,,,
95289,2014-01-01,Michigan,Muskegon,300 block of Monroe Avenue,0,0,http://www.gunviolencearchive.org/incident/95289,NaN,43.2301,NaN,-86.2514,bullets entered home,NaN,Adult 18+,Female,NaN,NaN,Unharmed,Victim
92401,2014-01-01,New Jersey,Newark,Central Avenue,0,0,http://www.gunviolencearchive.org/incident/92401,NaN,40.7417,NaN,-74.1695,Multiple detectives shot at once from vehicle ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92383,2014-01-01,New York,Queens,113th Avenue,1,0,http://www.gunviolencearchive.org/incident/92383,NaN,40.7034,NaN,-73.7474,Perpetrator is being sought by police.,"22,26","Adult 18+,Adult 18+","Male,Male","Julio Mora,Sheldon Smith",NaN,"Killed,Unharmed","Victim,Subject-Suspect"
92142,2014-01-01,New York,Brooklyn,St. Johns Place,0,1,http://www.gunviolencearchive.org/incident/92142,NaN,40.6715,NaN,-73.9476,NaN,34,"Adult 18+,Adult 18+","Male,Male",NaN,NaN,Injured,"Victim,Subject-Suspect"
95261,2014-01-01,Missouri,Springfield,Beverly Hills and Temple,0,1,http://www.gunviolencearchive.org/incident/95261,NaN,37.2646,NaN,-93.3007,NaN,"6,12","Child 0-11,Teen 12-17",Female,NaN,NaN,"Injured,Unharmed","Victim,Subject-Suspect"


In [8]:
df_gv.to_csv('data/gun_violence_clean.csv', index=False)

---
# States

In [9]:
df_state = pd.read_csv('data/nst-est2017-alldata.csv')
df_state.columns = [col.lower() for col in df_state.columns]
df_state.head()

,sumlev,region,division,state,name,census2010pop,estimatesbase2010,popestimate2010,popestimate2011,popestimate2012,popestimate2013,popestimate2014,popestimate2015,popestimate2016,popestimate2017,npopchg_2010,npopchg_2011,npopchg_2012,npopchg_2013,npopchg_2014,npopchg_2015,npopchg_2016,npopchg_2017,births2010,births2011,births2012,births2013,births2014,births2015,births2016,births2017,deaths2010,deaths2011,deaths2012,deaths2013,deaths2014,deaths2015,deaths2016,deaths2017,naturalinc2010,naturalinc2011,naturalinc2012,naturalinc2013,naturalinc2014,naturalinc2015,naturalinc2016,naturalinc2017,internationalmig2010,internationalmig2011,internationalmig2012,...,residual2010,residual2011,residual2012,residual2013,residual2014,residual2015,residual2016,residual2017,rbirth2011,rbirth2012,rbirth2013,rbirth2014,rbirth2015,rbirth2016,rbirth2017,rdeath2011,rdeath2012,rdeath2013,rdeath2014,rdeath2015,rdeath2016,rdeath2017,rnaturalinc2011,rnaturalinc2012,rnaturalinc2013,rnaturalinc2014,rnaturalinc2015,rnaturalinc2016,rnaturalinc2017,rinternationalmig2011,rinternationalmig2012,rinternationalmig2013,rinternationalmig2014,rinternationalmig2015,rinternationalmig2016,rinternationalmig2017,rdomesticmig2011,rdomesticmig2012,rdomesticmig2013,rdomesticmig2014,rdomesticmig2015,rdomesticmig2016,rdomesticmig2017,rnetmig2011,rnetmig2012,rnetmig2013,rnetmig2014,rnetmig2015,rnetmig2016,rnetmig2017
0,10.0,0,0,0.0,United States,308745538.0,308758105.0,309338421.0,311644280.0,313993272.0,316234505.0,318622525.0,321039839.0,323405935.0,325719178.0,580316.0,2305859.0,2348992.0,2241233.0,2388020.0,2417314.0,2366096.0,2313243.0,987836.0,3973485.0,3936976.0,3940576.0,3963195.0,3992376.0,3962714.0,3946000.0,598691.0,2512442.0,2501531.0,2608019.0,2582448.0,2699826.0,2728714.0,2744040.0,389145.0,1461043.0,1435445.0,1332557.0,1380747.0,1292550.0,1234000.0,1201960.0,191171.0,844816.0,913547.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.797410,12.585485,12.505244,12.485315,12.482760,12.298053,12.157903,8.091826,7.996742,8.276433,8.135526,8.441410,8.468405,8.454580,4.705584,4.588743,4.228811,4.349789,4.041351,3.829647,3.703323,2.720900,2.920371,2.883643,3.173228,3.516743,3.513394,3.423941,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.720900,2.920371,2.883643,3.173228,3.516743,3.513394,3.423941
1,20.0,1,0,0.0,Northeast Region,55317240.0,55318350.0,55388349.0,55642659.0,55860261.0,56047732.0,56203078.0,56296628.0,56359360.0,56470581.0,69999.0,254310.0,217602.0,187471.0,155346.0,93550.0,62732.0,111221.0,163452.0,646252.0,637891.0,635767.0,632435.0,634515.0,625336.0,621245.0,110732.0,470885.0,460988.0,479828.0,470012.0,488936.0,488935.0,493164.0,52720.0,175367.0,176903.0,155939.0,162423.0,145579.0,136401.0,128081.0,53255.0,242627.0,247800.0,...,-3278.0,-2551.0,-2739.0,-2369.0,-2428.0,-1183.0,-20.0,-560.0,11.640928,11.441691,11.362316,11.268248,11.280296,11.101691,11.012059,8.482045,8.268627,8.575402,8.374318,8.692218,8.680142,8.741722,3.158883,3.173065,2.786914,2.893930,2.588078,2.421549,2.270337,4.370437,4.444727,4.508096,4.808233,5.199160,5.311586,5.270640,-2.902486,-3.665590,-3.902223,-4.891065,-6.103092,-6.619089,-5.559570,1.467950,0.779137,0.605873,-0.082832,-0.903931,-1.307503,-0.288930
2,20.0,2,0,0.0,Midwest Region,66927001.0,66929794.0,66973360.0,67141501.0,67318295.0,67534451.0,67720120.0,67839187.0,67978168.0,68179351.0,43566.0,168141.0,176794.0,216156.0,185669.0,119067.0,138981.0,201183.0,212614.0,834915.0,830701.0,830859.0,836503.0,837962.0,829016.0,822534.0,140814.0,586823.0,584818.0,605237.0,597104.0,626496.0,610756.0,607187.0,71800.0,248092.0,245883.0,225622.0,239399.0,211466.0,218260.0,215347.0,23611.0,104806.0,114977.0,...,-1147.0,-319.0,-1140.0,-1351.0,-2842.0,-2662.0,-455.0,-1139.0,12.450745,12.356125,12.322463,12.369312,12.363032,12.207807,12.082094,8.751051,8.698779,8.976265,8.829336,9.243128,8.993784,8.918890,3.699694,3.657346,3.346198,3.539976,3.119904,3.214022,3.163204,1.562929,1.710206,1.706780,1.940799,2.134579,2.146559,2.113317,-2.75044

## To Do
* Drop
    * Rows: [1, 2, 3, 4, 13, 56, 57, 58]
* Keep
    * name
    * popestimate2014
    * popestimate2015
    * popestimate2016
    * popestimate2017
* Add
    * state abbreviation (code)
    * state capital lat/lon (center)
* Aggregate w/ Gun Violence (by year)
    * Killed
    * Injured

## Drop

In [10]:
df_state.drop([1, 2, 3, 4, 13, 56, 57, 58], inplace=True)

In [11]:
df_state = df_state[[
    'name',
    'popestimate2014',
    'popestimate2015',
    'popestimate2016',
    'popestimate2017'
]]
df_state.rename(columns={'name': 'state'}, inplace=True)

## Add

In [12]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'United States': 'US'
}

df_state['code'] = df_state['state'].apply(lambda state: us_state_abbrev[state])

In [13]:
capital_dict={
    'Alabama': 'Montgomery',
    'Alaska': 'Juneau',
    'Arizona':'Phoenix',
    'Arkansas':'Little Rock',
    # Geopy couldn't find Sacramento *shrug*
    'California': 'Los Angeles',
    'Colorado':'Denver',
    'Connecticut':'Hartford',
    'Delaware':'Dover',
    'Florida': 'Tallahassee',
    'Georgia': 'Atlanta',
    'Hawaii': 'Honolulu',
    'Idaho': 'Boise',
    'Illinois': 'Springfield',
    'Indiana': 'Indianapolis',
    'Iowa': 'Des Moines',
    'Kansas': 'Topeka',
    'Kentucky': 'Frankfort',
    'Louisiana': 'Baton Rouge',
    'Maine': 'Augusta',
    'Maryland': 'Annapolis',
    'Massachusetts': 'Boston',
    'Michigan': 'Lansing',
    'Minnesota': 'St. Paul',
    'Mississippi': 'Jackson',
    'Missouri': 'Jefferson City',
    'Montana': 'Helena',
    'Nebraska': 'Lincoln',
    'Nevada': 'Carson City',
    'New Hampshire': 'Concord',
    'New Jersey': 'Trenton',
    'New Mexico': 'Santa Fe',
    'New York': 'Albany',
    'North Carolina': 'Raleigh',
    'North Dakota': 'Bismarck',
    'Ohio': 'Columbus',
    'Oklahoma': 'Oklahoma City',
    'Oregon': 'Salem',
    'Pennsylvania': 'Harrisburg',
    'Rhode Island': 'Providence',
    'South Carolina': 'Columbia',
    'South Dakota': 'Pierre',
    'Tennessee': 'Nashville',
    'Texas': 'Austin',
    'Utah': 'Salt Lake City',
    'Vermont': 'Montpelier',
    'Virginia': 'Richmond',
    'Washington': 'Olympia',
    'West Virginia': 'Charleston',
    'Wisconsin': 'Madison',
    'Wyoming': 'Cheyenne'  
}

# for each state, looks up its capital and together are searched in a geocoder
# to return latitude and longitude of that city
centers = []
for state in df_state['state']:
    if state != 'United States':
        capital = capital_dict[state]
        results = geocoder.geocode(f'{capital} {state}')
        centers.append(f'{results.latitude},{results.longitude}')
    else:
        centers.append('')
df_state['center'] = centers
df_state.head()

,state,popestimate2014,popestimate2015,popestimate2016,popestimate2017,code,center
0,United States,318622525.0,321039839.0,323405935.0,325719178.0,US,
5,Alabama,4840037.0,4850858.0,4860545.0,4874747.0,AL,"32.3669656,-86.3006485"
6,Alaska,736759.0,737979.0,741522.0,739795.0,AK,"58.3019496,-134.419734"
7,Arizona,6706435.0,6802262.0,6908642.0,7016270.0,AZ,"33.4485866,-112.0773456"
8,Arkansas,2964800.0,2975626.0,2988231.0,3004279.0,AR,"34.7464809,-92.2895948"


## Aggregate

In [14]:
df_gv['year'] = df_gv['date'].apply(lambda x: x.year)

In [15]:
# Aggregate the deaths by year and by state
for year in [2014, 2015, 2016, 2017]:
    killed = []
    injured = []
    for state in df_state['state']:
        df_temp = df_gv[np.logical_and(df_gv['state']==state, df_gv['year']==year)]
        killed.append(df_temp['n_killed'].sum())
        injured.append(df_temp['n_injured'].sum())
    # Sum up United States
    killed[0] = sum(killed)
    injured[0] = sum(injured)
    
    df_state[f'killed{year}'] = killed
    df_state[f'injured{year}'] = injured

# Reorder columns
df_state = df_state[[
    'code', 'state', 'center', 'popestimate2014', 'popestimate2015', 'popestimate2016', 'popestimate2017', 
    'killed2014', 'killed2015', 'killed2016', 'killed2017', 
    'injured2014', 'injured2015', 'injured2016', 'injured2017'
]]

# Save

In [16]:
df_state.head()

,code,state,center,popestimate2014,popestimate2015,popestimate2016,popestimate2017,killed2014,killed2015,killed2016,killed2017,injured2014,injured2015,injured2016,injured2017
0,US,United States,,318622525.0,321039839.0,323405935.0,325719178.0,12465,13360,14960,15411,22704,26622,30241,30363
5,AL,Alabama,"32.3669656,-86.3006485",4840037.0,4850858.0,4860545.0,4874747.0,325,385,488,544,591,562,761,856
6,AK,Alaska,"58.3019496,-134.419734",736759.0,737979.0,741522.0,739795.0,29,70,88,69,49,84,103,70
7,AZ,Arizona,"33.4485866,-112.0773456",6706435.0,6802262.0,6908642.0,7016270.0,220,215,277,289,222,193,294,298
8,AR,Arkansas,"34.7464809,-92.2895948",2964800.0,2975626.0,2988231.0,3004279.0,169,155,190,216,231,245,351,437


In [17]:
# df_state.to_csv('data/state_clean.csv', index=False)